In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf

# Load and preprocess data with float32 type
data = pd.read_csv('E:\\chatbot-01\\datasets\\hr\\heart_rate_data.csv')
data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'])
data = data.drop(['date', 'time'], axis=1)
data = data.sort_values(by='datetime')

# Scale BPM values
scaler = MinMaxScaler(feature_range=(0, 1))
data['bpm_scaled'] = scaler.fit_transform(data[['bpm']])

# Add time-based features
data['hour'] = data['datetime'].dt.hour
data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24).astype('float32')
data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24).astype('float32')

# Define the sequence length and create sequences
sequence_length = 5

def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[['bpm_scaled', 'hour_sin', 'hour_cos']].values[i:i+sequence_length].astype('float32')
        sequences.append(seq)
    return np.array(sequences, dtype='float32')

# Create sequences
sequences = create_sequences(data, sequence_length)

# Split data into train-test
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(sequences, test_size=0.2, shuffle=False)

# Use smaller batch size
batch_size = 32

# Define the generator model
def build_generator(input_dim, sequence_length):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(sequence_length, input_dim)))
    model.add(Dropout(0.3))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dense(sequence_length, activation='sigmoid'))
    return model

generator = build_generator(input_dim=3, sequence_length=sequence_length)

# Define the WGAN discriminator model
def build_wgan_discriminator(sequence_length):
    model = Sequential()
    model.add(Input(shape=(sequence_length, 3)))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(1))  # No activation for WGAN
    return model

discriminator = build_wgan_discriminator(sequence_length)
discriminator.compile(loss='mean_squared_error', optimizer=RMSprop(learning_rate=0.0001))

# Build the WGAN
def build_wgan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(sequence_length, 3))
    generated_sequence = generator(gan_input)
    gan_output = discriminator(generated_sequence)
    gan = tf.keras.models.Model(gan_input, gan_output)
    gan.compile(loss='mean_squared_error', optimizer=RMSprop(learning_rate=0.0001))
    return gan

gan = build_wgan(generator, discriminator)

# Use a generator for training
def data_generator(data, batch_size, sequence_length):
    while True:
        idx = np.random.randint(0, len(data) - sequence_length, batch_size)
        batch_sequences = np.array([data[i:i+sequence_length] for i in idx], dtype='float32')
        yield batch_sequences, -np.ones((batch_size, 1))

train_gen = data_generator(X_train, batch_size, sequence_length)

# Train WGAN with data generator
epochs = 1000
steps_per_epoch = len(X_train) // batch_size

for epoch in range(epochs):
    # Train discriminator
    real_sequences = next(train_gen)[0]
    noise = np.random.normal(0, 1, (batch_size, sequence_length, 3))
    gen_sequences = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_sequences, -np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_sequences, np.ones((batch_size, 1)))

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, sequence_length, 3))
    g_loss = gan.train_on_batch(noise, -np.ones((batch_size, 1)))

    if epoch % 100 == 0:
        print(f"Epoch {epoch}/{epochs} - D loss: {0.5 * (d_loss_real + d_loss_fake)}, G loss: {g_loss}")


MemoryError: Unable to allocate 13.3 MiB for an array with shape (3, 579485) and data type float64